In [ ]:
import copy
from tqdm.notebook import tqdm

import numpy as np

import sys, os
sys.path.append('./pyFM/')
sys.path.append('./utils/')


import utils.baseline as baseline

import pyFM
from pyFM.mesh import TriMesh
import pyFM.spectral as spectral
import pyFM.eval
from functional_match import MultiShapeDifferenceMatching

In [ ]:
sys.path.append('./VisualizationTools/')
import VisualizationTools as plu

# 1. Load collection and ground truth correspondences 

In [ ]:
coll1 = [TriMesh(f'data/cats_lions/cat-{i:02d}.off').process() for i in tqdm(range(10))]
coll2 = [TriMesh(f'data/cats_lions/lion_cut1-{i:02d}.off').process() for i in tqdm(range(10))]


In [ ]:
maps1 = [np.arange(coll1[0].n_vertices) for _ in range(10)]
maps2 = [np.arange(coll2[0].n_vertices) for _ in range(10)]

In [ ]:
meshA = copy.deepcopy(coll1[0])
meshC = copy.deepcopy(coll2[0])

part2full = np.loadtxt("./data/cats_lions/lion_cut1_to_full", dtype=int)
full2full = np.loadtxt("./data/cats_lions/lion2cat", dtype=int)
    
gt_p2p = full2full[part2full]


s1_source = meshA.get_geodesic(force_compute=True, verbose=True)
sqrt_area = np.sqrt(meshA.area)

# def acc_eval(p2p, return_all=False):
#     if not return_all:
#         return pyFM.eval.accuracy(p2p, gt_p2p, s1_source, return_all=return_all)/sqrt_area
#     else:
#         acc, acc_dists = pyFM.eval.accuracy(p2p,gt_p2p,s1_source,return_all=return_all)
#         return acc/sqrt_area, acc_dists/sqrt_area

# 2. Matching using global embeddings

We here show the result of the main baseline on the collection.
The method solves

\begin{equation}
\min_{C\neq0} \sum_i \left( \|C V_i^1 - V_i^2 C\|^2 + \|C R_i^1 - R_i^2 C\|^2\right) + \alpha \|C\Delta^1- \Delta^2 C\|^2
\end{equation}

where $R$ and $V$ are the conformal and area shape difference operators, the index shows which deformation they embed and the exponent from which collections they belong.

This problem is presented in both

[1] Aharon Cohen and Mirela Ben-Chen. Robust Shape Collec-tion Matching and Correspondence from Shape Differences.Computer Graphics Forum, 39(2):555–568, May 2020

[2] Nitzan Shapira and Mirela Ben-Chen. Cross-Collection Map
Inference by Intrinsic Alignment of Shape Spaces. Computer
Graphics Forum, 33(5):281–290, Aug. 2014

In [ ]:
# Compute shape difference operators for each collection
SD_AB_list = baseline.get_SD_list(coll1, p2p=maps1, k1=50)
SD_CD_list = baseline.get_SD_list(coll2, p2p=maps2, k1=50)

In [ ]:
# You can play around the parameters to see. While the method works well for complete shape it can heavily fail
# in the presence of partiality
coll_match_options = {
    'alpha': 0.,
    'remove_area': False,
    'remove_conformal': False,
}

In [ ]:
FM_comm, p2p_comm = baseline.solve_collection(SD_AB_list[1:], SD_CD_list[1:], meshA, meshC, **coll_match_options)

In [ ]:
acc_comm = pyFM.eval.accuracy(p2p_comm, gt_p2p, s1_source, sqrt_area=sqrt_area)
print(f'Baseline accuracy : {1e3*acc_comm:.3f}')

In [ ]:
plu.plot_p2p(meshA, meshC, p2p_comm)

# 3. DWKS 

We solve the problem

\begin{equation}
\min_C E_d(C) + \alpha E_{dc}(C) + \beta E_{sd}(C)
\end{equation}

With $E_d$ the descriptor preservation term (using DWKS), $E_{dc}$ the descriptor commutativity term, $E_{sd}$ the commutativity with sahpe difference operators (like on the baseline).

Additionnaly, we give the opportunity to add a laplacian commutativity term (similarly to the baseline) as well as an orientation preserving term.

In [ ]:
process_options = {
    'n_ev': (50,50),
    'elims': (-np.log(3),np.log(3)), 
    'n_descr': 200,
    'scale': 1.4,
    'remove_area': True,
    'remove_conformal': False, 
    'SD_type': 'spectral',
    'mapping': list(zip(maps1, maps2)),
    'subsample_step': 2,
    'trim_values': True,
    'trim_scale': 2,
    'verbose': True
}

In [ ]:
model = MultiShapeDifferenceMatching(meshA, meshC)
model.preprocess(coll1[1:], coll2[1:], **process_options)

In [ ]:
fit_params = {
    'descr_mu': 1e0,  # Descriptor preservation
    'lap_mu': 0,  # Laplacian commutativity
    'descr_comm_mu': 1e1,  # Descriptor commutativity
    'SD_comm_mu': 1e-4,  # Shape Difference Commutativity
    'orient_mu': 0,  # Orientation preserving term
    'optinit': 'random',
    'verbose': True
}

In [ ]:
model.fit(**fit_params)

In [ ]:
p2p_fit = model.p2p
acc_fit = pyFM.eval.accuracy(p2p_fit, gt_p2p, s1_source, sqrt_area=sqrt_area)

print(f'Fitting DWKS : {1e3*acc_eval(p2p_fit):.3f}')

In [ ]:
plu.plot_p2p(meshA, meshC, p2p_fit, pretty=False)

One can see that the maps works kind of well (disambiguation of right and left). However, there remain some noise on the legs and around the cut especially.
Part of the noise can be removed  by ignoring worse matched vertices (compared with embedding distances) and projecting the map in low dimension. Be careful with partiality and dimension of the functional map

In [ ]:
ignore_ratio = .2
# Points to keep
dists, p2p = pyFM.spectral.knn_query(model.descr1, model.descr2, return_distance=True, n_jobs=-1)
subsample_C = np.nonzero(dists<=np.quantile(dists, 1-ignore_ratio))[0]
subsample_A = p2p_fit[subsample_C]

In [ ]:
dim_proj = (15,5)

In [ ]:
FM_ref = spectral.mesh_p2p_to_FM(np.arange(len(subsample_C)), meshA, meshC, dims=dim_proj, subsample=(subsample_A, subsample_C))
p2p_ref = spectral.FM_to_p2p(FM_ref, meshA.eigenvectors, meshC.eigenvectors)

In [ ]:
plu.plot_p2p(meshA, meshC, p2p_ref)

This looks like a good starting point for a refinement algorithm. Sadly there is no good refinement algorithm tailored for partial shapes so one has to play around with parameters of isometric refinement algorithm.